In [7]:
import psycopg2
import pandas as pd
conn = psycopg2.connect("dbname=postgres host=db user=postgres password=postgres")
conn.autocommit = True
cursor = conn.cursor()

Each time a rocket is launched, one or more cores (first stages) are involved. Sometimes, cores are recovered after the launch and reused posteriorly in another launch. What is the maximum number of times a core has been used? Write an SQL query to find the result.

In [11]:
query = """
SELECT core_id, count(DISTINCT launch_id)
FROM public.cores
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1
"""
cursor.execute(query)
result = cursor.fetchall()
pd.DataFrame(result, columns=['core_id', 'launch_count'])


,core_id,launch_count
0,5e9e28a7f3591817f23b2663,14


Which cores have been reused in less than 50 days after the previous launch? Write an SQL query to find the result.


In [15]:
query = """
WITH prev_dates AS (
    SELECT 
        core_id,
        reused,
        l.date_utc,
        LEAD(l.date_utc) OVER (PARTITION BY core_id ORDER BY l.date_local DESC) AS previous_date_utc
    FROM public.cores c
    LEFT JOIN public.launches l
           ON c.launch_id = l.launch_id
), reused_limit AS (
SELECT 
    core_id, 
    date_utc,
    previous_date_utc,
    DATE_PART('DAY', date_utc - previous_date_utc) AS date_diff,
    CASE WHEN DATE_PART('DAY', date_utc - previous_date_utc) < 50 THEN True ELSE False END is_under_reused_limit
FROM prev_dates
)
SELECT core_id, date_utc, previous_date_utc, date_diff
FROM reused_limit
WHERE is_under_reused_limit
"""

cursor.execute(query)
result = cursor.fetchall()
pd.DataFrame(result, columns=['core_id', 'date_utc', 'previous_date_utc', 'date_diff'])

,core_id,date_utc,previous_date_utc,date_diff
0,5e9e28a6f359183c413b265d,2022-09-05 02:09:00,2022-08-04 23:08:00,31.0
1,5e9e28a6f359183c413b265d,2022-03-09 13:45:00,2022-01-31 23:11:12,36.0
2,5e9e28a6f35918c0803b265c,2021-01-20 13:02:00,2020-12-13 17:30:00,37.0
3,5e9e28a7f3591817f23b2663,2022-02-21 14:44:00,2022-01-13 15:25:00,38.0
4,5e9e28a7f3591817f23b2663,2021-05-15 22:54:00,2021-04-07 16:34:00,38.0
5,5e9e28a7f3591817f23b2663,2021-04-07 16:34:00,2021-03-11 08:13:00,27.0
6,5e9e28a7f3591817f23b2663,2021-03-11 08:13:00,2021-01-24 15:00:00,45.0
7,5e9e28a7f3591817f23b2663,2021-01-24 15:00:00,2020-12-06 16:17:00,48.0
8,5ef670f10059c33cee4a826c,2022-06-01 17:08:50,2022-04-21 15:16:00,41.0
9,5ef670f10059c33cee4a826c,2022-04-21 15:16:00,2022-03-03 14:35:00,49.0


In [13]:
query = """
SELECT 
    DATE_TRUNC('MONTH', date_utc) AS month,
    count(DISTINCT launch_id) AS launches
FROM public.launches
GROUP BY 1
HAVING count(DISTINCT launch_id) > 1
ORDER BY 2 DESC
"""
cursor.execute(query)
result = cursor.fetchall()
pd.DataFrame(result, columns=['month', 'launches'])

,month,launches
0,2022-11-01,8
1,2022-04-01,6
2,2022-08-01,6
3,2022-07-01,6
4,2021-12-01,5
5,2022-06-01,5
6,2022-12-01,5
7,2022-05-01,5
8,2021-06-01,4
9,2020-11-01,4
